# QCoDeS Example with the Rigol DS 1074 Z oscilloscope

This notebook presents the implementation of QCoDeS driver for the Rigol DS1074Z oscilloscope.

## Importing dependencies 

In [1]:
import visa

#Qcodes import
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.rigol.DS1074Z import DS1074Z
from qcodes.instrument.parameter import expand_setpoints_helper
from qcodes import initialise_database, load_or_create_experiment, Measurement

## Create the instrument 

The address of the oscilloscope is needed to create the instrument. The address can also be found using the NI-MAX explorer.

The instrument is created in the following way: 

In [2]:
rigol = DS1074Z('rigol', 'USB0::0x1AB1::0x04CE::DS1ZB161650342::INSTR')

Connected to: RIGOL TECHNOLOGIES DS1074Z (serial:DS1ZB161650342, firmware:00.04.00) in 5.41s


## Trigger setup  

Trigger source can be set to any channel (1 to 4). Here we use the input signal from channel 1 itself as a source for trigger. 

In [4]:
rigol.trigger_edge_source('ch1')
rigol.trigger_edge_slope('negative')

The trigger-mode type supported by this oscilloscope are `EDGe,PULSe, SLOPe, VIDeo, PATTern, DURATion`. Both the trigger mode and trigger level can be set in the following manner. 

In [5]:
rigol.trigger_mode('edge')
rigol.trigger_level(0.1)

## Data acquisition and plotting

The data acquisition is performed using measurement context manager. After initializing the database and loading/creating the experiment, the parameters are registered. The result are added to the database via datasaver object. The plot can be easily obtained using ``plot_by_id``.

Here, we show two ways of data acquisition. First, where setpoint is defined upfront (while registering the parameter and second, where a helper function is used to acquire both the oscilloscope trace and its setpoint. 

In [ ]:
initialise_database()
exp = load_or_create_experiment(experiment_name='Oscilloscope trace',
                               sample_name='no_name')

In [ ]:
meas = Measurement()
trace = rigol.channels.ch1.trace
time = rigol.time_axis
meas.register_parameter(time)
meas.register_parameter(trace, setpoints=(time,))

with meas.run() as datasaver:
        trace_val = trace()
        time_val = time()
        datasaver.add_result((time, time_val),
                             (trace, trace_val))


In [ ]:
_ = plot_dataset(datasaver.dataset)

Now, the same measurement can also be performed using a helper function `expand_setpoints_helper`. This function takes a ParameterWithSetpoints and acquires the parameter along with it's setpoints. The data is returned in a format prepared to insert into the dataset. Parameter `trace` is a ParameterWithSetpoints. 

In [ ]:
meas_helper = Measurement()
meas_helper.register_parameter(rigol.channels.ch1.trace)

with meas_helper.run() as datasaver:
        trace_val = trace()
        time_val = time()
        datasaver.add_result(*expand_setpoints_helper(rigol.channels.ch1.trace))

In [ ]:
_ = plot_dataset(datasaver.dataset)